# Image Classification with MobileNetV2 and Transfer Learning

## Import Required Libraries

## Define Paths and Hyperparameters

In [ ]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
DATA_DIR = './dataset_2'
TEST_DIR = './test'

In [ ]:
# Load Datasets (Grayscale)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode="grayscale"
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode="grayscale"
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    color_mode="grayscale"
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)

Found 480 files belonging to 4 classes.
Using 384 files for training.
Found 480 files belonging to 4 classes.
Using 96 files for validation.
Found 480 files belonging to 4 classes.


In [ ]:
# Optimise performance with prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Data Augmentation (applied during training)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

## Data Augmentation & Preprocessing

In [ ]:
# Load Base Model (EfficientNetB0, pretrained on ImageNet)
# Input shape must have 3 channels, so grayscale images will be broadcast to match
base_model = EfficientNetB0(include_top=False, input_shape=(*IMG_SIZE, 3), weights='imagenet')
base_model.trainable = True  # Fine-tune entire model

In [ ]:
# Assemble full model with classifier on top
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),  # Preprocess for EfficientNet
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Compute class weights to balance class distribution
y_train = np.concatenate([y.numpy() for _, y in train_ds])
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weight_dict = dict(enumerate(class_weights))

2025-06-06 15:33:27.742587: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight_dict
)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.4066 - loss: 1.3182 - val_accuracy: 0.2708 - val_loss: 1.4586
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7100 - loss: 0.8960 - val_accuracy: 0.3229 - val_loss: 1.3354
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8048 - loss: 0.6753 - val_accuracy: 0.4167 - val_loss: 1.1517
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8610 - loss: 0.4556 - val_accuracy: 0.5833 - val_loss: 0.9162
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9071 - loss: 0.3605 - val_accuracy: 0.7292 - val_loss: 0.6621
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9401 - loss: 0.2700 - val_accuracy: 0.8021 - val_loss: 0.5217
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9496 - loss: 0.2033 - val_accuracy: 0.8333 - val_loss: 0.4172
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9540 - loss: 0.1997 - val_accuracy: 0.9062 - val_loss:

## Evaluate the Model

In [14]:
# 🧪 Predict and evaluate performance
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step


2025-06-06 15:39:02.351593: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# 📊 Display metrics
print("Test Accuracy:", np.mean(np.array(y_true) == np.array(y_pred)))
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:
# 💾 Save the trained model
model.save('mri_classifier.keras')

## Early Stopping Callback

## Classification Report

In [ ]:
# Print the classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

## Confusion Matrix

In [ ]:
# Generate and plot the confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.show()

## Evaluate the Model on the Test Set

In [ ]:
# Reset the test generator and make predictions
test_generator.reset()
preds = model.predict(test_generator)

# Convert the predicted probabilities to class labels
y_pred = np.argmax(preds, axis=1)

# True labels from the test generator
y_true = test_generator.classes

## Visualise Sample Images from the Dataset

In [ ]:
import matplotlib.pyplot as plt
class_names = test_generator.class_indices
inv_class_names = {v: k for k, v in class_names.items()}

# Show first 9 images from test set
plt.figure(figsize=(10, 10))
for i in range(9):
    img, label = test_generator[i]
    plt.subplot(3, 3, i + 1)
    plt.imshow(img[0].astype('uint8'))
    plt.title(f"Label: {inv_class_names[np.argmax(label[0])]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## Visualise Misclassified Samples

In [ ]:
# Show first 9 misclassified examples
pred_labels = np.argmax(preds, axis=1)
true_labels = test_generator.classes
misclassified_idx = np.where(pred_labels != true_labels)[0]

plt.figure(figsize=(10, 10))
for i, idx in enumerate(misclassified_idx[:9]):
    img, _ = test_generator[idx]
    plt.subplot(3, 3, i + 1)
    plt.imshow(img[0].astype('uint8'))
    plt.title(f"True: {class_labels[true_labels[idx]]}, Pred: {class_labels[pred_labels[idx]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()